<a href="https://colab.research.google.com/github/githubvishwas/lasso_cae_ml/blob/main/Exercise-2_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install lasso-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.8 MB 1.5 MB/s 
     |████████████████████████████████| 237 kB 60.8 MB/s 
     |████████████████████████████████| 51 kB 6.1 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=cdb0cf4937dab5e23c725f72b9ab89f597a49b7b4a28401b2baf78847bcd2593
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [4]:
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


In [19]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import glob
import os
import ntpath
from ipywidgets import Image, Layout, interact, interactive, fixed, interact_manual, widgets
from lasso.dyna import D3plot, ArrayType
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding, MDS
from google.colab import output

In [8]:
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'lasso_python_ml','exercise-1','d3plots')
filepath_list = glob.glob(DATADIR + "/*.d3plot")
print(f"Found {len(filepath_list)} files.")

Found 100 files.


In [20]:
output.enable_custom_widget_manager()

In [9]:
# let's collect node displacement from all d3plots
fields = []

for i_filepath, filepath in enumerate(filepath_list):
    
    # print progress
    print(f"filepath {filepath} ({i_filepath + 1}/{len(filepath_list)})", end='\r')
    
    # load d3plot
    d3plot = D3plot(filepath)
    
    # extract node displacement at last timestep
    node_displacement = d3plot.arrays[ArrayType.node_displacement]
    fields.append(node_displacement)

# print end
print(f"Done ({i_filepath + 1}/{len(filepath_list)})")

Done (100/100)


In [10]:
# convert list into a numpy array
data = np.stack(fields)

n_simulations, n_timesteps, n_nodes, xyz = data.shape
print(f"(n_simulations {n_simulations}, n_timesteps {n_timesteps}, n_nodes {n_nodes}, xyz {xyz})")

(n_simulations 100, n_timesteps 5, n_nodes 2500, xyz 3)


In [11]:
# select some simulation
some_simulation = 0

# transform shape
data_sim = data[some_simulation].reshape((n_timesteps, n_nodes*xyz))

print(f"(n_timesteps {data_sim.shape[0]}, n_nodes*xyz flattened {data_sim.shape[1]})")

(n_timesteps 5, n_nodes*xyz flattened 7500)


In [12]:
u, s, v = np.linalg.svd(data_sim,full_matrices=False)
print(f"u {u.shape}")
print(f"s {s.shape}")
print(f"v {v.shape}")

u (5, 5)
s (5,)
v (5, 7500)


In [13]:
# display singular values/eigenvalues
fig = go.FigureWidget()
fig.add_trace(
    go.Scatter(x=np.arange(len(s)), 
               y=s, 
               mode='lines+markers',
               name="Singular Values"
              )
)

fig.update_layout(
    title='Singular Values',
    xaxis_title='Eigenvalue Index',
    yaxis_title='Eigenvalue',
)

fig.show()

In [21]:
i_timestep = 0
i_modes = 1
def reconstruct_simulation(i_modes):
    d = u[:,:i_modes] @ np.diag(s[:i_modes]) @ v[:i_modes, :]
    return d.reshape((n_timesteps, n_nodes, xyz))

data_reconstructed = reconstruct_simulation(i_modes)[i_timestep]
#print(f"Reconstruction accuracy: mode - {i_modes}: ",np.std(t - data_reconstructed))

fig = go.FigureWidget(
    data=[
        go.Mesh3d(
            x=data_reconstructed[:, 0],
            y=data_reconstructed[:, 1],
            z=data_reconstructed[:, 2],
            #mode='markers',
            name="Embedding Simulation",
            #marker={
            #   "size": 5
            #},
            #text=filepath_list,
            #hoverinfo="text",
            #opacity=0.99
        )
    ]
)

# some styling 
fig.update_layout(title='3D Embedding of Simulations',
                  margin=go.layout.Margin(
                    l=0, #left margin
                    r=0, #right margin
                    b=0, #bottom margin
                    t=0  #top margin
                  ),
                  #hovermode="closest",
                  autosize=True,
                  scene = dict(
                  xaxis = dict(nticks=4, range=[-5,105]),
                  yaxis = dict(nticks=4, range=[-5,20]),
                  zaxis = dict(nticks=4, range=[-1,5]),),
                 )

def on_timestep_change(timestep):
    global i_timestep
    i_timestep = timestep
    data_reconstructed[:] = reconstruct_simulation(i_modes)[i_timestep]
    fig.update_traces(overwrite=True, 
                      x=data_reconstructed[:, 0],
                      y=data_reconstructed[:, 1],
                      z=data_reconstructed[:, 2],)

def on_modes_change(mode):
    global i_modes
    i_modes = mode
    data_reconstructed[:] = reconstruct_simulation(i_modes)[i_timestep]
    fig.update_traces(overwrite=True, 
                      x=data_reconstructed[:, 0],
                      y=data_reconstructed[:, 1],
                      z=data_reconstructed[:, 2],)



interact(on_timestep_change, timestep=widgets.IntSlider(min=0, max=n_timesteps-1, step=1, value=0))
interact(on_modes_change, mode=widgets.IntSlider(min=1, max=len(s), step=1, value=1))

print(data_reconstructed.shape)
fig


interactive(children=(IntSlider(value=0, description='timestep', max=4), Output()), _dom_classes=('widget-inte…

interactive(children=(IntSlider(value=1, description='mode', max=5, min=1), Output()), _dom_classes=('widget-i…

(2500, 3)


FigureWidget({
    'data': [{'name': 'Embedding Simulation',
              'type': 'mesh3d',
              'ui…

In [16]:

output.enable_custom_widget_manager()

In [17]:
from google.colab import output
output.disable_custom_widget_manager()